In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import re
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


# پیش‌بینی هشتگ در توییتر با استفاده از Naive Bayes و CAR

برای این تمرین از سه روش مختلف برای پیش‌بینی هشتگ‌های مناسب برای توییت‌ها استفاده شده است:

1. الگوریتم Naive Bayes کتابخانه sklearn  بر اساس بردارهای تعبیه

2. الگوریتم Naive Bayes بر اساس احتمالات

3. قوانین  (Class Association Rules - CAR)




## پیش پردازش


دیتای مورد استفاده در این تمرین، پیش‌تر در تکلیف قبلی اماده شده است؛ مجموعه‌داده‌ی مربوطه به‌صورت یک فایل سی اس وی بوده و شامل متن ۵۰۰۰ توییت می‌باشد


In [3]:
texts_df = pd.read_csv('./texts.csv')

In [4]:
texts_df.head()

,Unnamed: 0,text,preprocess_texts
0,0,هدف من پیدا کردن ایران ستیزان بود که شما رو یافتم \r\nحالا شما از پشت لباس پادشاهی دوست بیا بیرون چون طرفداران پادشاهی… https://t.co/ZKXhxrtVZU,هدف من پیدا کردن ایران ستیزان بود که شما رو یافتم حالا شما از پشت لباس پادشاهی دوست بیا بیرون چون طرفداران پادشاهی
1,1,@reise_ghabile ظهر شمام بخیر,ظهر شمام بخیر
2,2,دوست پسر خلبان کجا میفروخن؟,دوست پسر خلبان کجا میفروخن
3,3,@HoseinMr007 خب اونجا زندگی میکنیم,خب اونجا زندگی می‌کنیم
4,4,RT @debianstable1: صبحا که میخوان منو از خواب بیدار کنن... https://t.co/ZOLx1KOqTW,صبحا که میخوان منو از خواب بیدار کنن


### تابع استخراج هشتگ‌ها

`extract_hashtags`: این تابع هشتگ‌ها را از متن توییت استخراج می‌کند
- ورودی: متن توییت
- خروجی: لیستی از هشتگ‌های موجود در متن
- از عبارات منظم (regex) برای پیدا کردن کلمات بعد از علامت # استفاده می‌کند


In [5]:
def extract_hashtags(text):
    """Extract hashtags from text"""
    return re.findall(r'#(\w+)', text)


### تابع  پیش‌پردازش متن

`preprocess_text`: این تابع متن توییت را پاک‌سازی و نرمال‌سازی می‌کند
- حذف لینک‌ها
- حذف کاراکترهای خاص
- حذف فاصله‌های اضافی
- حذف منشن‌ها (@username)
- نگهداری فاصله

In [6]:
def preprocess_text(text):
    """Clean and preprocess text"""
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\.\S+|(\S\.com)+','', text) 
     # Remove mentions and RT(retweet) lables
    text = re.sub(r'@[\w]+', '', text)
    text = re.sub(r'(rt :)+|[؟.!="،]+|[@\r\n\t()*&^\%$!:…rt]+', '', text)
    # Remove special characters
    text = re.sub(r'[^\u0621-\u0628\u062A-\u063A\u0641-\u0642\u0644-\u0648\u064E-\u0651\u0655\u067E\u0686\u06A9\u06AF\u06BE\u06CC\u200C\u200F ]+$/', '', text)
    # Remove extra spaces
    text = re.sub(r'[ ]{2,}', ' ', text)
    return text.strip()

### انجام پیش پردازش متن و استخراج هشتگ ها بر روی دیتا
   - هشتگ‌های اصلی هر توییت (`hashtags`)
   - متن پیش‌پردازش شده (`clean_text`)
   - متن توییت‌ها بدون هشتگ (`removed_hashtags`)


In [7]:
texts_df['clean_text'] = texts_df['text'].apply(preprocess_text)
texts_df['hashtags'] = texts_df['text'].apply(extract_hashtags)
texts_df['has_hashtag'] = texts_df['hashtags'].apply(lambda x: len(x) > 0)
texts_df['removed_hashtags'] = texts_df['clean_text'].apply(lambda x: re.sub(r'#(\w+)', '', x))

# Split data into tweets with and without hashtags
tweets_with_hashtags = texts_df[texts_df['has_hashtag']]
tweets_without_hashtags = texts_df[~texts_df['has_hashtag']]
print(f"There are {len(tweets_with_hashtags)} tweets with hashtags")

There are 1337 tweets with hashtags


در   داده‌ها، ۱۳۳۷ توییت با هشتگ وجود دارد. براساس انچه در تمرین خواسته شده، ۲۰٪ از آن‌ها یعنی ۲۶۷ توییت  برای آموزش انتخاب  می‌شود


In [8]:
train_size = 0.20
tweets_with_hashtags_train = tweets_with_hashtags.sample(frac=train_size, random_state=42)

In [9]:
tweets_with_hashtags.head()

,Unnamed: 0,text,preprocess_texts,clean_text,hashtags,has_hashtag,removed_hashtags
7,7,RT @Ftm_Jahani: #صداوسيما پاسخ گو باشد\r\nبرای رفتار غیر اخلاقی کارشناس آقای خسروی چه دارید بگویید؟\r\nدر خصوص این اهانت باید از اقای #سعید_محمد…,صداوسیما پاسخ‌گو باشدبرای رفتار غیر اخلاقی کارشناس آقای خسروی چه دارید بگوییددر خصوص این اهانت باید از اقای سعید محمد,#صداوسيما پاسخ گو باشدبرای رفتار غیر اخلاقی کارشناس آقای خسروی چه دارید بگوییددر خصوص این اهانت باید از اقای #سعید_محمد,"[صداوسيما, سعید_محمد]",True,پاسخ گو باشدبرای رفتار غیر اخلاقی کارشناس آقای خسروی چه دارید بگوییددر خصوص این اهانت باید از اقای
8,8,نمیدونم چندمین رای میشه\r\nI vote #Dynamite for #BestMusicVideo at the #iHeartAwards \r\n@BTS_twt,نمیدونم چندمین رای میشهi voe dynamie fo besmusicvideo a he iheaawads,نمیدونم چندمین رای میشهi voe #dynamie fo #besmusicvideo a he #iheaawads,"[Dynamite, BestMusicVideo, iHeartAwards]",True,نمیدونم چندمین رای میشهi voe fo a he
10,10,RT @Mozafari_hadi70: از جهت وجه خارجی هم انتخابات و مشارکت مردم نشان دهنده اقتدار ملی است.\r\n#دیار_صددرصد_رای_آری,از جهت وجه خارجی هم انتخابات و مشارکت مردم نشان‌دهنده اقتدار ملی است دیار صددرصد رای آری,از جهت وجه خارجی هم انتخابات و مشارکت مردم نشان دهنده اقتدار ملی است#دیار_صددرصد_رای_آری,[دیار_صددرصد_رای_آری],True,از جهت وجه خارجی هم انتخابات و مشارکت مردم نشان دهنده اقتدار ملی است
11,11,RT @sarbazetwitt: فوری\r\nدکتر #سعید_محمد طی تماس تلفنی از #سعید_جلیلی جهت ثبت نام در انتخابات دعوت بعمل آورد. https://t.co/9NkIFcfwEH,فوریدکتر سعید محمد طی تماس تلفنی از سعید جلیلی جهت ثبت‌نام در انتخابات دعوت بعمل آورد,فوریدکتر #سعید_محمد طی تماس تلفنی از #سعید_جلیلی جهت ثبت نام در انتخابات دعوت بعمل آورد,"[سعید_محمد, سعید_جلیلی]",True,فوریدکتر طی تماس تلفنی از جهت ثبت نام در انتخابات دعوت بعمل آورد
13,13,با چراغی همه جا گشتم وگشتم در شهر هیچ کس ... هیچ کس اینجا به تو مانند نشد #وحدت_برای_مردم #دوباره_ایثار #قالیباف https://t.co/zMPCx2GmlV,با چراغی همه‌جا گشتم وگشتم در شهر هیچ‌کس هیچ‌کس اینجا به تو مانند نشد وحدت برای مردم دوباره ایثار قالیباف,با چراغی همه جا گشتم وگشتم در شهر هیچ کس هیچ کس اینجا به تو مانند نشد #وحدت_برای_مردم #دوباره_ایثار #قالیباف,"[وحدت_برای_مردم, دوباره_ایثار, قالیباف]",True,با چراغی همه جا گشتم وگشتم در شهر هیچ کس هیچ کس اینجا به تو مانند نشد


In [10]:
tweets_without_hashtags.head()

,Unnamed: 0,text,preprocess_texts,clean_text,hashtags,has_hashtag,removed_hashtags
0,0,هدف من پیدا کردن ایران ستیزان بود که شما رو یافتم \r\nحالا شما از پشت لباس پادشاهی دوست بیا بیرون چون طرفداران پادشاهی… https://t.co/ZKXhxrtVZU,هدف من پیدا کردن ایران ستیزان بود که شما رو یافتم حالا شما از پشت لباس پادشاهی دوست بیا بیرون چون طرفداران پادشاهی,هدف من پیدا کردن ایران ستیزان بود که شما رو یافتم حالا شما از پشت لباس پادشاهی دوست بیا بیرون چون طرفداران پادشاهی,[],False,هدف من پیدا کردن ایران ستیزان بود که شما رو یافتم حالا شما از پشت لباس پادشاهی دوست بیا بیرون چون طرفداران پادشاهی
1,1,@reise_ghabile ظهر شمام بخیر,ظهر شمام بخیر,ظهر شمام بخیر,[],False,ظهر شمام بخیر
2,2,دوست پسر خلبان کجا میفروخن؟,دوست پسر خلبان کجا میفروخن,دوست پسر خلبان کجا میفروخن,[],False,دوست پسر خلبان کجا میفروخن
3,3,@HoseinMr007 خب اونجا زندگی میکنیم,خب اونجا زندگی می‌کنیم,خب اونجا زندگی میکنیم,[],False,خب اونجا زندگی میکنیم
4,4,RT @debianstable1: صبحا که میخوان منو از خواب بیدار کنن... https://t.co/ZOLx1KOqTW,صبحا که میخوان منو از خواب بیدار کنن,صبحا که میخوان منو از خواب بیدار کنن,[],False,صبحا که میخوان منو از خواب بیدار کنن


## مدل ها

## <p>پیاده‌سازی مدل <span dir="ltr">Naive Bayse</span> با تعبیه‌سازی کلمات</p>
<p>در این بخش ابتدا یک   <span dir="ltr">CountVectorizer</span> از کتابخانه  <span dir="ltr">sklearn</span>  ایجاد میشود که یرای تعبیه کردن استفاده میشود، سپس در متغیر  <span dir="ltr">`y_train`</span> لیبل ها قرار دارند، اولین هشتگ به عنوان لیبل انتخاب می‌شود
</p>
و سپس مدل اماده کتابخانه اموزش داده می‌شود.

In [11]:
# Prepare data for Naive Bayes
vectorizer = CountVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(tweets_with_hashtags_train['clean_text'])

# Create labels from hashtags (using the first hashtag for simplicity)
y_train = tweets_with_hashtags_train['hashtags'].apply(lambda x: x[0] if x else '')

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)


MultinomialNB()

`predict_hashtags_nb`: تابع پیش‌بینی هشتگ با Naive Bayes
- ورودی: متن توییت
- خروجی: سه هشتگ پیشنهادی با بالاترین احتمال

<p>در این قطعه کد، ابتدا متن ورودی با استفاده از <span dir="ltr">vectorizer</span> به یک نمایش برداری تبدیل می‌شود. سپس تابع <span dir="ltr">predict_proba</span> از مدل <span dir="ltr">nb_model</span> فراخوانی می‌شود تا احتمال تعلق متن به هر یک از کلاس‌ها (هشتگ‌ها) محاسبه گردد. در ادامه، این احتمالات مرتب‌سازی می‌شوند و شاخص‌های مربوط به <span dir="ltr">top_n</span> هشتگ با بالاترین احتمال انتخاب می‌شوند. در نهایت، با استفاده از این شاخص‌ها، نام هشتگ‌های متناظر از لیست کلاس‌های مدل (<span dir="ltr">nb_model.classes_</span>) استخراج شده و به‌عنوان خروجی بازگردانده می‌شوند.</p>

In [12]:
def predict_hashtags_nb(text, top_n=3):
    """Predict top N hashtags using Naive Bayes"""
    text_vector = vectorizer.transform([text])
    probs = nb_model.predict_proba(text_vector)
    top_indices = np.argsort(probs[0])[-top_n:][::-1]
    return [nb_model.classes_[i] for i in top_indices]

## <p> پیاده سازی الگوریتم <span dir="ltr">Naive Bayse</span> با کلاس و احتساب احتمالات  </p>

در پیاده‌سازی الگوریتم ناییو بیس، دو نوع احتمال اصلی محاسبه می‌شود: 

1. **احتمال پیشین**: نشان‌دهنده فراوانی نسبی هر هشتگ در داده‌های آموزشی است.
2. **احتمال شرطی**: نشان می‌دهد که یک کلمه خاص چقدر احتمال دارد در متونی که به یک هشتگ خاص تعلق دارند، ظاهر شود. 

این احتمالات به مدل کمک می‌کنند تا هشتگ‌های مناسب برای متون جدید را پیش‌بینی کند.

##### ساختار کلاس
در این پیاده‌سازی، سه متغیر اصلی تعریف شده است:
- `word_probs`: دیکشنری برای نگهداری احتمال شرطی هر کلمه برای هر هشتگ.
- `hashtag_probs`: دیکشنری برای نگهداری احتمال پیشین هر هشتگ.
- `vocab`: مجموعه‌ای از تمام کلمات مشاهده شده در متون.
- `word_counts`: دیکشنری برای نگهداری تعداد تکرار کلمات برای هر هشتگ.
- `hashtag_counts`: دیکشنری برای نگهداری تعداد تکرار هر هشتگ.

#### توابع 

#### تابع پیش‌پردازش (`preprocess`)

در این تابع، متن ورودی به کلمات جداگانه تقسیم می‌شود:
- متن به حروف کوچک تبدیل می‌گردد
- بر اساس فاصله‌ها به کلمات مجزا تفکیک می‌شود
- لیستی از کلمات برگردانده می‌شود

#### تابع `fit`


**ورودی‌ها:**
1. **متون آموزشی**: مجموعه‌ای از متون که هر کدام به یک یا چند هشتگ مرتبط هستند.
2. **هشتگ‌های آموزشی**: لیستی از هشتگ‌ها که به هر متن آموزشی اختصاص داده شده‌اند.

**فرآیند:**
1. **شمارش و محاسبات اولیه**:
   - **تعداد تکرار کلمات**: برای هر هشتگ، تعداد تکرار هر کلمه در متون مرتبط با آن هشتگ محاسبه می‌شود. این کار به مدل کمک می‌کند تا بفهمد کدام کلمات برای هر هشتگ مهم‌تر هستند.
   - **تعداد تکرار هشتگ‌ها**: تعداد دفعاتی که هر هشتگ در مجموعه داده‌ها ظاهر می‌شود، شمارش می‌گردد.
   - **به‌روزرسانی واژگان**: کلمات جدیدی که در متون آموزشی ظاهر می‌شوند به مجموعه واژگان مدل اضافه می‌شوند.

2. **محاسبه احتمالات**:
   - **احتمال پیشین هشتگ‌ها**: این احتمال نشان‌دهنده فراوانی نسبی هر هشتگ در کل مجموعه داده‌ها است و با تقسیم تعداد تکرار هر هشتگ بر کل تعداد هشتگ‌ها محاسبه می‌شود.
   - **احتمال شرطی کلمات**: برای هر کلمه در هر هشتگ، احتمال شرطی آن کلمه با استفاده از هموارسازی لاپلاس محاسبه می‌شود. این احتمال نشان می‌دهد که یک کلمه خاص چقدر احتمال دارد در متونی که به یک هشتگ خاص تعلق دارند، ظاهر شود.

 **خروجی:**
- **مدل آموزش‌دیده**: پس از اجرای این فرآیند، مدل بیز ساده آماده است تا با استفاده از احتمالات محاسبه‌شده، متون جدید را طبقه‌بندی کند و هشتگ‌های مناسب را پیش‌بینی نماید.

#### تابع پیش‌بینی (`predict`)

در این تابع، هشتگ‌های مناسب برای متن جدید پیش‌بینی می‌شوند:

۱. **محاسبه امتیاز**:
   - برای هر هشتگ، لگاریتم احتمال پیشین محاسبه می‌گردد
   - برای هر کلمه در متن، لگاریتم احتمال شرطی به امتیاز اضافه می‌شود
   - از لگاریتم برای جلوگیری از سرریز عددی استفاده می‌شود

۲. **انتخاب هشتگ‌ها**:
   - هشتگ‌ها بر اساس امتیاز مرتب می‌شوند
   - تعداد مشخصی از بهترین هشتگ‌ها برگردانده می‌شوند

#### Source Code of Naive Bayse in class implementation:

**نکات مهم پیاده‌سازی**
- از هموارسازی لاپلاس برای جلوگیری از احتمالات صفر استفاده شده است یعنی وقتی مدل با کلمه جدیدی روبرو میشود تعداد کلمه جدید در دیتای ترین را به جای صفر یک در نظر میگیریم
- محاسبات در فضای لگاریتمی انجام می‌شود تا از مشکلات عددی جلوگیری شود
- امکان تنظیم تعداد هشتگ‌های پیشنهادی فراهم شده است

In [13]:
from collections import Counter
class CustomNaiveBayes:
    def __init__(self):
        self.word_probs = {}  # احتمال هر کلمه برای هر هشتگ
        self.hashtag_probs = {}  # احتمال پیشین هر هشتگ
        self.vocab = set()  # مجموعه تمام کلمات
        self.word_counts = {}  # نگهداری تعداد تکرار کلمات برای هر هشتگ
        self.hashtag_counts = {}  # نگهداری تعداد تکرار هر هشتگ
            
    def get_hashtag_frequency(self, hashtag):
        """برگرداندن تعداد تکرار یک هشتگ در داده‌ها"""
        return self.hashtag_counts.get(hashtag, 0)

    def get_statistics(self):
        """
        نمایش آمار کلی از داده‌های آموزشی
        """
        stats = {
            'total_hashtags': len(self.hashtag_probs),
            'total_unique_words': len(self.vocab),
            'hashtag_word_counts': {},
            'top_words_per_hashtag': {},
        }
        
        # محاسبه تعداد کلمات برای هر هشتگ
        for hashtag in self.word_counts:
            stats['hashtag_word_counts'][hashtag] = {
                'total_words': sum(self.word_counts[hashtag].values()),
                'unique_words': len(self.word_counts[hashtag]),
                'frequency': self.hashtag_counts[hashtag]
            }            
            # پیدا کردن پرتکرارترین کلمات برای هر هشتگ
            sorted_words = sorted(
                self.word_counts[hashtag].items(), 
                key=lambda x: x[1], 
                reverse=True
            )[:5]
            stats['top_words_per_hashtag'][hashtag] = sorted_words
            
        print(stats)
        return stats
    
    def print_statistics(self):
        """
        چاپ آمار به صورت خوانا
        """
        stats = self.get_statistics()
        print("=== آمار کلی ===")
        print(f"تعداد کل هشتگ‌های یکتا: {stats['total_hashtags']}")
        print(f"تعداد کل کلمات یکتا: {stats['total_unique_words']}")
        
        print("\n=== آمار هر هشتگ ===")
        # مرتب‌سازی هشتگ‌ها بر اساس فراوانی
        sorted_hashtags = sorted(
            stats['hashtag_word_counts'].items(),
            key=lambda x: x[1]['frequency'],
            reverse=True
        )
        print(sorted_hashtags)
        for hashtag, counts in sorted_hashtags:
            print(f"\nهشتگ: {hashtag}")
            print(f"تعداد تکرار: {counts['frequency']}")
            print(f"تعداد کل کلمات: {counts['total_words']}")
            print(f"تعداد کلمات یکتا: {counts['unique_words']}")
            print("پرتکرارترین کلمات:")
            for word, count in stats['top_words_per_hashtag'][hashtag]:
                print(f"    {word}: {count}")
    
    def preprocess(self, text):
        """تبدیل متن به لیستی از کلمات"""
        words = text.lower().split()
        return words
    
    def fit(self, texts, hashtags):
        """آموزش مدل"""
        self.word_counts = {}  # {hashtag: {word: count}}
        self.hashtag_counts = {}  # {hashtag: count}
        print(f"Number of texts in train data: {len(texts)}")
        print(f"Number of hashtags in train data: {len(hashtags)}")
        for text, hashtag in zip(texts, hashtags):
            if not hashtag:  # اگر هشتگ خالی است
                continue
            # جدا سازی و اپدیت کلمات جدید در وکب
            words = self.preprocess(text)
            self.vocab.update(words)
            # شمارش هشتگ‌ها
            self.hashtag_counts[hashtag] = self.hashtag_counts.get(hashtag, 0) + 1
            # شمارش کلمات برای هر هشتگ
            if hashtag not in self.word_counts:
                self.word_counts[hashtag] = {}
            # حساب کلمات هر کلاس
            for word in words:
                if not word:  # اگر کلمه خالی است
                    continue
                self.word_counts[hashtag][word] = self.word_counts[hashtag].get(word, 0) + 1
        print(f"Number of unique hashtags: {len(self.hashtag_counts)}")
        print(f"Number of hashtags with word counts: {len(self.word_counts)}")
        # محاسبه احتمالات
        total_hashtags = sum(self.hashtag_counts.values())
        # احتمال پیشین هر هشتگ
        for hashtag, count in self.hashtag_counts.items():
            self.hashtag_probs[hashtag] = count / total_hashtags
        # احتمال هر کلمه برای هر هشتگ (با هموارسازی لاپلاس)
        for hashtag in self.hashtag_counts:
            self.word_probs[hashtag] = {}
            total_words = sum(self.word_counts[hashtag].values()) + len(self.vocab)
            for word in self.vocab:
                count = self.word_counts[hashtag].get(word, 0) + 1  # هموارسازی لاپلاس
                self.word_probs[hashtag][word] = count / total_words
    
    def predict(self, text, top_n=3):
        """پیش‌بینی هشتگ‌های مناسب"""
        words = self.preprocess(text)
        scores = {}
        
        # محاسبه امتیاز هر هشتگ
        for hashtag in self.hashtag_probs:
            # شروع با لگاریتم احتمال پیشین
            score = np.log(self.hashtag_probs[hashtag])
            
            # اضافه کردن لگاریتم احتمال کلمات
            for word in words:
                if word in self.vocab:
                    score += np.log(self.word_probs[hashtag].get(word, 1/len(self.vocab)))
            
            scores[hashtag] = score
        
        # انتخاب بهترین هشتگ‌ها
        sorted_hashtags = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        return [h[0] for h in sorted_hashtags[:top_n]]
    

In [14]:
custom_nb = CustomNaiveBayes()
train_texts = tweets_with_hashtags_train['removed_hashtags']
train_hashtags = tweets_with_hashtags_train['hashtags'].apply(lambda x: x[0] if x else '')
custom_nb.fit(train_texts, train_hashtags)

Number of texts in train data: 267
Number of hashtags in train data: 267
Number of unique hashtags: 120
Number of hashtags with word counts: 120


In [15]:
def predict_hashtags_custom_nb(text, top_n=4):      
    return custom_nb.predict(text, top_n)

### پیاده‌سازی قوانین انجمنی کلاس (CAR)
`generate_car_rules`: این تابع قوانین انجمنی را تولید می‌کند
- پارامترها:
  - min_support: حداقل پشتیبانی برای قوانین (پیش‌فرض: 0.001)
  - min_confidence: حداقل اطمینان برای قوانین (پیش‌فرض: 0.5)
- مراحل:
  1. تبدیل متن‌ها به تراکنش‌ها
  2. محاسبه مجموعه کلمات پرتکرار
  3. تولید قوانین با اطمینان بالا


تولید قوانین انجمنی کلاس (`generate_car_rules`):
- تبدیل متون به تراکنش‌ها: هر متن به مجموعه‌ای از کلمات تبدیل می‌شود و هشتگ‌های مربوط به آن به عنوان برچسب در نظر گرفته می‌شوند
- تولید مجموعه‌های اقلام پرتکرار: تعداد تکرار هر کلمه در تراکنش‌ها محاسبه می‌شود و کلماتی که فراوانی آن‌ها از حداقل پشتیبانی تعیین‌شده بیشتر است، انتخاب می‌شوند.
- تولید قوانین: برای هر کلمه پرتکرار، احتمال وقوع آن با هر هشتگ محاسبه می‌شود. اگر اطمینان این احتمال از حداقل اطمینان تعیین‌شده بیشتر باشد، قانون مربوطه به لیست قوانین اضافه می‌شود.

In [16]:
def generate_car_rules(texts_df, min_support=0.001, min_confidence=0.7):
    """Generate Class Association Rules"""
    # Convert texts to transactions
    transactions = []
    labels = []
    for _, row in texts_df.iterrows():
        words = set(row['clean_text'].split())
        hashtags = set(row['hashtags'])
        if hashtags:
            transactions.append(words)
            labels.append(list(hashtags)[0])  # Use first hashtag as label
    # Generate frequent itemsets
    word_counts = {}
    for transaction in transactions:
        for word in transaction:
            word_counts[word] = word_counts.get(word, 0) + 1
    # Filter by support
    frequent_words = {word: count for word, count in word_counts.items() 
                     if count/len(transactions) >= min_support}
    # Generate rules
    rules = []
    for word in frequent_words:
        for label in set(labels):
            support_word = sum(1 for t in transactions if word in t)
            support_label = sum(1 for l in labels if l == label)
            support_both = sum(1 for t, l in zip(transactions, labels) 
                             if word in t and l == label)
            
            confidence = support_both / support_word if support_word > 0 else 0
            
            if confidence >= min_confidence:
                rules.append((word, label, confidence))
    return rules
car_rules = generate_car_rules(tweets_with_hashtags_train)

In [17]:
car_rules[:10]

[('اهالی', 'رستم_قاسمی', 1.0),
 ('منتظرت', 'رستم_قاسمی', 1.0),
 ('فارس', 'رستم_قاسمی', 1.0),
 ('آرمی', 'BestMusicVideo', 1.0),
 ('#besmusicvideo', 'BestMusicVideo', 0.8),
 ('عقبیمi', 'BestMusicVideo', 1.0),
 ('میدونستین', 'BestMusicVideo', 1.0),
 ('#dynamie', 'BestMusicVideo', 0.8),
 ('تئوری', 'محسن_رضایی', 1.0),
 ('اقتصادی', 'محسن_رضایی', 0.7777777777777778)]

پیش‌بینی هشتگ‌ها با استفاده از قوانین انجمنی کلاس (`predict_hashtags_car`):
- تبدیل متن به مجموعه کلمات: متن ورودی با اسپلیت به مجموعه‌ای از کلمات تبدیل می‌شود
- پیش‌بینی هشتگ‌ها: برای هر کلمه در متن، اگر در قوانین موجود باشد، هشتگ‌های مربوطه با اطمینان آن‌ها به لیست پیش‌بینی‌ها اضافه می‌شود.
- مرتب‌سازی و انتخاب بهترین‌ها: پیش‌بینی‌ها بر اساس اطمینان مرتب شده و تعداد مشخصی از بهترین هشتگ‌ها برگردانده می‌شود

In [18]:
def predict_hashtags_car(text, rules, top_n=3):
    """Predict top N hashtags using CAR rules"""
    words = set(text.split())
    predictions = []
    
    for word, label, confidence in rules:
        if word in words:
            predictions.append((label, confidence))
    
    # Sort by confidence and return top N
    predictions.sort(key=lambda x: x[1], reverse=True)
    return [pred[0] for pred in predictions[:top_n]]


## مقایسه و ارزیابی و تحلیل نهایی 

`compare_methods`: مقایسه نتایج دو روش
- نمایش متن اصلی
- نمایش پیش‌بینی‌های Naive Bayes
- نمایش پیش‌بینی‌های CAR

In [19]:
def compare_methods(test_text , original_text,original_tweet_tupple):
    """Compare hashtag predictions from both methods"""
    print("Original text:", original_text)
    print(f"Original Hashtag: {original_tweet_tupple[1]['hashtags'][0]}")
    print(f"\nNumber of original hashtag in train data: {custom_nb.get_hashtag_frequency(original_tweet_tupple[1]['hashtags'][0])}")
    print("\n\t- Naive Bayes predictions:", predict_hashtags_custom_nb(test_text))
    print("\n\t- Naive Bayes predictions Vectorized(Sklearn Model):", predict_hashtags_nb(test_text))
    print("\n\t- CAR predictions:", predict_hashtags_car(preprocess_text(test_text), car_rules))
    print("================================================")


In [20]:
test_tweets = tweets_with_hashtags.sample(n=5)
test_tweets_without_hashtags = test_tweets['removed_hashtags']
for tweet_text , original_tweet_tupple in zip(test_tweets_without_hashtags, list(test_tweets.iterrows())):
     compare_methods(tweet_text, original_tweet_tupple[1]['clean_text'] , original_tweet_tupple)

Original text: امروز رژیم آخوندی در چنان تنگنای مرگباری گرفتار شده که در تاریخچه این رژیم به ‌ندرت سابقه دارد#ایران #ianegimechang
Original Hashtag: ایران

Number of original hashtag in train data: 2

	- Naive Bayes predictions: ['محسن_رضایی', 'SabaHungerStrike', 'رئیسی_متشکریم', 'آقامحسن_ایران']

	- Naive Bayes predictions Vectorized(Sklearn Model): ['محسن_رضایی', 'SabaHungerStrike', 'رئیسی_متشکریم']

	- CAR predictions: ['سیرک_انتخابات']
Original text: مرد اقدام و تحول از هیچ تلاش و تکاپویی برای حل مشکلات اقشار ضعیف و آسیب پذیر فروگذار نخواهد کرد#اقدام_و_تحول#محسن_رض
Original Hashtag: اقدام_و_تحول

Number of original hashtag in train data: 10

	- Naive Bayes predictions: ['محسن_رضایی', 'آقامحسن_ایران', 'رئیسی_متشکریم', 'اقدام_و_تحول']

	- Naive Bayes predictions Vectorized(Sklearn Model): ['محسن_رضایی', 'SabaHungerStrike', 'اقدام_و_تحول']

	- CAR predictions: ['رئیسی_متشکریم', 'رئیسی_متشک']
Original text: #سعید_محمد #دولت_جوان_انقلابی تا اخر ایستاده ایم 🇮🇷🇮🇷
Original Hashtag: سعید_مح

### **تحلیل**
اولا بنابر ارتباط مستقیم تعداد نمونه ها در داده اموزشی تعداد هشتگ ها در تست ها تعداد هشتگ مربوطه در داده اموزشی نشان داده شده است

نتایج نشان می‌دهد که عملکرد مدل‌های پیش‌بینی هشتگ بسته فراوانی هشتگ در داده‌های آموزشی متفاوت است. در مواردی که هشتگ اصلی دارای فراوانی بالایی در داده‌های آموزشی بوده و یا ارتباط معنایی قوی با کلمات موجود در متن دارد، دقت پیش‌بینی‌ها افزایش می‌یابد. در مقابل، توییت هایی که هشتگ‌های آن‌ها در داده‌های آموزشی نادر هستند یا از نظر معنایی ارتباط مستقیمی با کلمات متن ندارند (خصوصا در مدل ناییو بیس با وکتورها)، چالش‌برانگیز بوده و منجر به پیش‌بینی‌های نادرستی می‌شوند.
پس چون مدل های ناییو بیس، براساس احتمالات و فراوانی کلمات هستند، در مواجهه با کلمات جدید ضعیف عمل می کنند.

<p> مدل  <span dir="ltr">CAR</span>  نیز زمانی عملکرد مطلوبی دارد که کلمات کلیدی مشخصی در توییت تست وجود داشته باشند که در داده‌های آموزشی، قوانین قوی و با اطمینان بالایی را برای یک هشتگ خاص ایجاد کرده باشند. </p>


- ✅ هشتگ اول SabaHungerStrike : هشتگ موردنظر در داده‌ی آموزشی با فراوانی زیادی (۲۲ بار) مشاهده شده است پس احتمال پریدکت صحیح هشتگ توسظ هر سه مدل به‌درستی توسط انجام شده است.
- ❌ سوم هشتگ 엑소:  در این مورد اما هشتگ موردنظر با فراوانی پایین (۵ بار) در داده‌ی آموزشی وجود داشته که می تواند دلیل اصلی خطا در پیش بینی باشد همچنین متن شامل واژگان غیر فارسی است که در مدل وکتورکردن داده ها نیز ممکن است احتمال خظا را بالا ببرد.


In [ ]:
def evaluate_methods(test_data):
    """Evaluate both methods on test data"""
    nb_correct = car_correct = nb_correct_vectorized= total = 0
    
    for _, row in test_data.iterrows():
        true_hashtags = set(row['hashtags'])
        if not true_hashtags:
            continue
            
        nb_predictions = set(predict_hashtags_custom_nb(row['clean_text']))
        nb_predictions_vectorized = set(predict_hashtags_nb(row['clean_text']))
        car_predictions = set(predict_hashtags_car(row['clean_text'], car_rules))
        
        nb_correct += len(true_hashtags.intersection(nb_predictions))
        nb_correct_vectorized += len(true_hashtags.intersection(nb_predictions_vectorized))
        car_correct += len(true_hashtags.intersection(car_predictions))
        total += len(true_hashtags)
    
    print("Naive Bayes Accuracy:", nb_correct/total if total > 0 else 0)
    print("Naive Bayes Accuracy Vectorized(Sklearn function):", nb_correct_vectorized/total if total > 0 else 0)
    print("CAR Accuracy:", car_correct/total if total > 0 else 0)

# Evaluate on remaining tweets with hashtags
test_data = tweets_with_hashtags[~tweets_with_hashtags.index.isin(tweets_with_hashtags_train.index)]
evaluate_methods(test_data)

Naive Bayes Accuracy: 0.368270332187858
Naive Bayes Accuracy Vectorized(Sklearn function): 0.43585337915234823
CAR Accuracy: 0.28121420389461627
